In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix, roc_curve, auc, plot_roc_curve

In [5]:
path_train = "dataset/train"
path_test = "dataset/test"

img_train = tf.keras.preprocessing.image_dataset_from_directory(
    path_train,
    validation_split=0.2,
    subset = "training",
    seed = 125,
    image_size = (224,224),
    batch_size = 32
)

img_validation = tf.keras.preprocessing.image_dataset_from_directory(
    path_train,
    validation_split=0.2,
    subset = "validation",
    seed = 125,
    image_size = (224,224),
    batch_size = 32
)


Found 22145 files belonging to 3 classes.
Using 17716 files for training.
Found 22145 files belonging to 3 classes.
Using 4429 files for validation.


In [3]:
def metrics(true, pred):
    accuracy=accuracy_score(true, pred)
    precision=precision_score(true, pred,average='weighted')
    f1Score=f1_score(true, pred, average='weighted')
    cm=confusion_matrix(true, pred)
    sensGlacoma = cm[0][0]/(cm[0][0]+cm[0][1]+cm[0][2])
    specGlaucoma = (cm[1][1]+cm[1][2]+cm[2][1]+cm[2][2])/(cm[1][1]+cm[1][2]+cm[2][1]+cm[2][2]+cm[1][0]+cm[2][0])
    sensNormal = cm[1][1]/(cm[1][0]+cm[1][1]+cm[1][2])
    specNormal = (cm[0][0]+cm[0][2]+cm[2][0]+cm[2][2])/(cm[0][1]+cm[2][1]+cm[0][0]+cm[0][2]+cm[2][0]+cm[2][2])
    sensOther = cm[2][2]/(cm[0][2]+cm[1][2]+cm[2][2])
    specOther = (cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1])/(cm[0][2]+cm[1][2]+cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1])
    
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    print("sensGlacoma : {}".format(sensGlacoma))
    print("specGlaucoma : {}".format(specGlaucoma))
    print("sensNormal : {}".format(sensNormal))
    print("specNormal : {}".format(specNormal))
    print("sensOther : {}".format(sensOther))
    print("specOther : {}".format(specOther))
    
    print(cm)
    return accuracy, precision, f1Score, sensGlacoma, specGlaucoma, sensNormal, specNormal, sensOther, specOther

In [3]:
VGG16_MODEL=tf.keras.applications.VGG16(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')
VGG16_MODEL.trainable=False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(3,activation='softmax')

In [4]:
model = tf.keras.Sequential([
  VGG16_MODEL,
  global_average_layer,
  prediction_layer
])

model.compile(optimizer="adam", 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 1539      
Total params: 14,716,227
Trainable params: 1,539
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
history = model.fit(img_train,
                    epochs=18, 
                    steps_per_epoch=30,
                    validation_steps=2,
                    validation_data=img_validation)
model.save("deeplearningVGG16.h5")

Epoch 1/18
30/30 [==============================] - 118s 4s/step - loss: 0.6899 - accuracy: 0.6781 - val_loss: 0.6326 - val_accuracy: 0.6875
Epoch 2/18
30/30 [==============================] - 120s 4s/step - loss: 0.6645 - accuracy: 0.7010 - val_loss: 0.7267 - val_accuracy: 0.6406
Epoch 3/18
30/30 [==============================] - 120s 4s/step - loss: 0.6849 - accuracy: 0.6896 - val_loss: 0.6065 - val_accuracy: 0.6719
Epoch 4/18
30/30 [==============================] - 121s 4s/step - loss: 0.6852 - accuracy: 0.6792 - val_loss: 0.4717 - val_accuracy: 0.8125
Epoch 5/18
30/30 [==============================] - 119s 4s/step - loss: 0.6557 - accuracy: 0.7115 - val_loss: 0.5703 - val_accuracy: 0.7188
Epoch 6/18
30/30 [==============================] - 121s 4s/step - loss: 0.6824 - accuracy: 0.6979 - val_loss: 0.6243 - val_accuracy: 0.6562
Epoch 7/18
30/30 [==============================] - 121s 4s/step - loss: 0.6630 - accuracy: 0.6906 - val_loss: 0.6916 - val_accuracy: 0.7031
Epoch 8/18
30

In [23]:
img_test = tf.keras.preprocessing.image_dataset_from_directory(
    path_test,
    image_size = (224,224)
)


label = ["glaucoma","normal","other"]

loadmodel = tf.keras.models.load_model("deeplearningVGG16.h5")

predict = loadmodel.predict(img_test)
prediction = np.argmax(predict)

#redictions = np.array([])
#labels =  np.array([])
#for x, y in img_test:
    #predictions = np.concatenate([predictions,np.argmax(loadmodel.predict(x), axis=-1)])
    #labels = np.concatenate([labels, np.argmax(y.numpy(), axis = -1)])



#print(predict)

Found 4470 files belonging to 3 classes.


TypeError: only integer scalar arrays can be converted to a scalar index

In [10]:
valAcc, valPrec, valFScore, valsensGlaucoma, valspecGlaucoma, valsensNormal, valspecNormal, valsensOther, valspecOther = metrics(label, predictions)

TypeError: Singleton array 12830 cannot be considered a valid collection.

In [22]:
i = 0
glau = 0
norm = 0
oth = 0
for i in predict:
    index = np.argmax(i)
    if index == 0:
        glau+=1
    elif index == 1:
        norm+=1
    elif index==2 :
        oth+=1
    
print(glau)
print(norm)
print(oth)

1977
1619
874


In [18]:
#print(len(predict))
print(predict[0])

[0.9277266  0.05035294 0.02192038]


In [11]:
import imghdr
import cv2
import os
import glob


for file in glob.glob('dataset/test/other/*.jpg'):
    image = cv2.imread(file)
    file_type = imghdr.what(file)
    if file_type != 'jpeg':
        print(file + " - invalid - " + str(file_type))
        cv2.imwrite(file, image)
    print("finish jpg",image)

dataset/test/other\other1007.jpg - invalid - png
finish jpg [[[2 1 3]
  [2 1 3]
  [2 1 3]
  ...
  [2 2 2]
  [2 2 2]
  [2 2 2]]

 [[2 1 3]
  [2 1 3]
  [2 1 3]
  ...
  [2 2 2]
  [2 2 2]
  [2 2 2]]

 [[2 1 3]
  [2 1 3]
  [2 1 3]
  ...
  [2 2 2]
  [2 2 2]
  [2 2 2]]

 ...

 [[2 2 2]
  [2 2 2]
  [2 2 2]
  ...
  [2 1 3]
  [2 1 3]
  [2 1 3]]

 [[2 2 2]
  [2 2 2]
  [2 2 2]
  ...
  [2 1 3]
  [2 1 3]
  [2 1 3]]

 [[2 2 2]
  [2 2 2]
  [2 2 2]
  ...
  [2 1 3]
  [2 1 3]
  [2 1 3]]]
dataset/test/other\other1019.jpg - invalid - png
finish jpg [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
dataset/test/other\other1034.jpg - invalid - png
fin